In [1]:
import os
import numpy as np
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Initialize the pre-trained MobileNet model without the top classification layer
model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Function to extract features from an image
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Preprocess for MobileNet
    features = model.predict(img_array)
    return features.flatten()

# Directory paths
sushi_train_dir = '../data/food11/train/sushi'
sushi_test_dir = '../data/food11/test/sushi'

# Collect all image paths in the pizza folders
image_paths = [os.path.join(sushi_train_dir, fname) for fname in os.listdir(sushi_train_dir)]
image_paths += [os.path.join(sushi_test_dir, fname) for fname in os.listdir(sushi_test_dir)]

# Extract features for all images
features = []
for img_path in image_paths:
    try:
        features.append(extract_features(img_path, model))
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")

features = np.array(features)
print(f"Extracted features for {len(features)} images")

Extracted features for 1000 images


In [2]:
from sklearn.cluster import KMeans

# Number of clusters (choose based on expected subcategories, e.g., 3 types of pizza)
num_clusters = 7

# Perform K-Means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(features)

# Get cluster labels for each image
cluster_labels = kmeans.labels_

# Print the cluster label for each image
for img_path, label in zip(image_paths, cluster_labels):
    print(f"{img_path} assigned to cluster {label}")

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


../data/food11/train/sushi\1005352.jpg assigned to cluster 1
../data/food11/train/sushi\1012499.jpg assigned to cluster 4
../data/food11/train/sushi\1013418.jpg assigned to cluster 6
../data/food11/train/sushi\1016390.jpg assigned to cluster 1
../data/food11/train/sushi\1018953.jpg assigned to cluster 5
../data/food11/train/sushi\1022922.jpg assigned to cluster 3
../data/food11/train/sushi\1025041.jpg assigned to cluster 0
../data/food11/train/sushi\1029831.jpg assigned to cluster 0
../data/food11/train/sushi\1032052.jpg assigned to cluster 3
../data/food11/train/sushi\1032351.jpg assigned to cluster 5
../data/food11/train/sushi\1033302.jpg assigned to cluster 5
../data/food11/train/sushi\1049178.jpg assigned to cluster 2
../data/food11/train/sushi\1057384.jpg assigned to cluster 3
../data/food11/train/sushi\1058864.jpg assigned to cluster 4
../data/food11/train/sushi\1060699.jpg assigned to cluster 0
../data/food11/train/sushi\1063569.jpg assigned to cluster 0
../data/food11/train/sus

In [3]:
import shutil

# Create directories for each cluster (subcategories)
for i in range(num_clusters):
    train_subdir = os.path.join(sushi_train_dir, f"cluster_{i}")
    test_subdir = os.path.join(sushi_test_dir, f"cluster_{i}")
    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(test_subdir, exist_ok=True)

# Move images to subdirectories based on cluster labels
for img_path, label in zip(image_paths, cluster_labels):
    if 'train' in img_path:
        shutil.move(img_path, os.path.join(sushi_train_dir, f"cluster_{label}", os.path.basename(img_path)))
    elif 'test' in img_path:
        shutil.move(img_path, os.path.join(sushi_test_dir, f"cluster_{label}", os.path.basename(img_path)))

print("Images moved to their respective cluster directories.")

Images moved to their respective cluster directories.


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Fine-grained apple_pie classification
sushi_train_gen = ImageDataGenerator(rescale=1./255)
sushi_test_gen = ImageDataGenerator(rescale=1./255)

sushi_train_generator = sushi_train_gen.flow_from_directory(
    sushi_train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

sushi_test_generator = sushi_test_gen.flow_from_directory(
    sushi_test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Use the same base model (MobileNet)
base_model_sushi = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model_sushi.layers:
    layer.trainable = False

# Add custom layers for apple_pie classification
x_sushi = base_model_sushi.output
x_sushi = GlobalAveragePooling2D()(x_sushi)
x_sushi = Dense(512, activation='relu')(x_sushi)
sushi_predictions = Dense(num_clusters, activation='softmax')(x_sushi)

# Create and compile the model
sushi_model = Model(inputs=base_model_sushi.input, outputs=sushi_predictions)
sushi_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the fine-grained apple_pie classification model
sushi_model.fit(
    sushi_train_generator,
    steps_per_epoch=sushi_train_generator.samples // 32,
    validation_data=sushi_test_generator,
    validation_steps=sushi_test_generator.samples // 32,
    epochs=30
)

# Save the model
sushi_model.save('fine_grained_sushi_classifier.h5')

Found 900 images belonging to 7 classes.
Found 100 images belonging to 7 classes.
Epoch 1/30
28/28 [==============================] - 9s 276ms/step - loss: 1.7708 - accuracy: 0.3237 - val_loss: 1.3574 - val_accuracy: 0.5729
Epoch 2/30
28/28 [==============================] - 7s 255ms/step - loss: 1.0325 - accuracy: 0.6809 - val_loss: 0.9574 - val_accuracy: 0.7083
Epoch 3/30
28/28 [==============================] - 7s 259ms/step - loss: 0.6874 - accuracy: 0.8260 - val_loss: 0.7489 - val_accuracy: 0.7708
Epoch 4/30
28/28 [==============================] - 7s 257ms/step - loss: 0.5061 - accuracy: 0.9018 - val_loss: 0.6690 - val_accuracy: 0.7917
Epoch 5/30
28/28 [==============================] - 7s 252ms/step - loss: 0.3905 - accuracy: 0.9366 - val_loss: 0.5341 - val_accuracy: 0.8646
Epoch 6/30
28/28 [==============================] - 7s 247ms/step - loss: 0.3215 - accuracy: 0.9631 - val_loss: 0.5175 - val_accuracy: 0.8333
Epoch 7/30
28/28 [==============================] - 7s 260ms/step 